### Use this notebook for developing mlframe prototype 2

In [1]:
import multiprocessing
from Queue import Queue, Empty
import subprocess
import os
import time
import re
import sys

In [26]:
# Class for executing commands in background processes.
# command_and_args can be a list or a string with the command and arguments separated with spaces.
# It can be used for executing remote processes in background. For that purpose exec_remote.sh should be used.
# Usage sample: 
# d = BashExecutor(command,hostname=hostname)
# d.start()
# exec_remote.sh must output subprocess exit code in the form:
# exitcode=N
# , where N is the number.
class BashExecutor(multiprocessing.Process):
    
    def __init__(self, command_and_args, debug=False):
        super(BashExecutor,self).__init__()
        self.debug = debug
        if self.debug: print "command args:",command_and_args
        if type(command_and_args) is str:
            self.command = command_and_args.split(" ")
        else:
            self.command=command_and_args
        self.exitcode_pat = re.compile("^exitcode=(\d+)")
        manager = multiprocessing.Manager()
        d = manager.dict()
        self.ec = None
        # Two flags below for correct detection 
        # when all output from external process was read.
        d["stdout_empty"] = False
        d["stderr_empty"] = False 
        d[1]="one"
        d["stdout"] = ""
        d["stderr"] = ""
        self.d = d
        
        
    # Poll exit code of self.proc and store it if not None.
    def poll(self):
        exitcode = self.proc.poll()
#         if self.debug: 
#             if exitcode is not None:
#                 print "polled exitcode=",exitcode
        if exitcode is not None:
            self.setExitCode(exitcode)
        if self.debug: print "<",self.d["stdout_empty"],self.d["stderr_empty"],">"
        return exitcode
    
    
    # Set given exit code to Command class object
    def setExitCode(self, ec):
        if "exitcode" not in self.d:
            if self.debug: print "setting exit code to",ec        
            self.d["exitcode"] = ec
            if False: #isinstance(self.command, Command): 
                if self.command.exitcode is None:
                    self.command.exitcode = ec
                if self.debug:
                    print "Set exit code to",ec
                    if ec is not None:
                        printObj(self.command)
    
    
    # Copy process stdout to self.stdout queue line by line.
    def __handleOutput__(self, q):
        proc = self.proc   
        if q.full():
            print "%",
        for line in iter(proc.stdout.readline, b''):
            if line is not None:
                m = self.exitcode_pat.match(line)
                if m is not None:
                    print "Exitcode match",m.group(0)
                q.put(line)
                self.d["stdout"] += line
                if self.debug: 
                    print ">",line,
                #sys.stdout.flush()
                time.sleep(1)
            
            #if line is not None and len(line)>0:
            #        print self.hostname,line,
        if self.debug: 
            print "Exiting handleOutput",line,"."
            out,err = proc.communicate()
            print "out:",out
            print "err:",err
        self.d["stdout_empty"] = True
                    
    # Copy process stderr to self.stderr queue line by line.
    def __handleError__(self, q):
        proc = self.proc
        if q.full():
            print "#"
        for line in iter(proc.stderr.readline, b''):
            if line is not None:
                q.put(line)
            else:
                self.d["stderr_empty"] = True
            
        self.d["stderr_empty"] = True
            #if line is not None and len(line)>0:
            #        print "!",self.hostname,line,
            #sys.stdout.flush()
        
    # Must be calles to start the process in background
    def run(self):
        if self.debug: 
            print "In {}. Calling {}".format(self.name,self.command)
            
        proc = subprocess.Popen(self.command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=1000, shell=False)
        self.proc = proc
        out_q = multiprocessing.Queue(5)
        p_out = multiprocessing.Process(target=self.__handleOutput__,args=(out_q,))
        p_out.start()
        err_q = multiprocessing.Queue(5)
        p_err = multiprocessing.Process(target=self.__handleError__,args=(err_q,))
        p_err.start()
        if self.debug: print self.name, "process started"
        while self.poll() is None or not self.d["stdout_empty"] or not self.d["stderr_empty"]:
            if self.debug: 
                print "(",self.poll(),self.d["stdout_empty"],self.d["stderr_empty"],")"
            
            try:
                outline = out_q.get_nowait()
                if self.debug:
                    if outline is not None:
                        print "OUT",outline,
                    
            except Empty:
                if self.debug: print "-",
                time.sleep(.1)
            
            try:
                errline = err_q.get_nowait()
                if self.debug:
                    if errline is not None:
                        print "ERR", errline,
            except Empty:
                if self.debug: print "=",
                time.sleep(.1)
            
        try:
            line = out_q.get_nowait()                
            if self.debug:
                print "STDOUT",line
        except Empty:
            pass
        
        if self.debug:
            print "Exiting",multiprocessing.current_process().name,self.d["exitcode"]
        return

    def getExitcode(self):
        if self.debug: print "Should be one:",self.d[1]
        return self.d["exitcode"]

In [27]:
d1 = BashExecutor("/Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh",debug=True)
d1.start()
#d2 = RemoteExec("mouse","test_nvidia_ubuntu.sh", debug=True)
#RemoteExec("DL","test_nvidia_ubuntu.sh", debug=False)

command args: /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh
In BashExecutor-23. Calling ['/Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh']
BashExecutor-23 process started
< False False >
( < False False >
> Running test command on peters-macbookpro.local
None False False )
OUT Running test command on peters-macbookpro.local
= < False False >
( < False False >
None False False )
- = < False False >
( < False False >
None False False )
- ERR peters-macbookpro.local err1
< False False >
( < False False >
None False False )
- = < False False >
( < False False >
None False False )
- = setting exit code to 125
< False True >
( < False True >
125 False True )
- ERR peters-macbookpro.local err2
> peters-macbookpro.local():1
< False True >
( < False True >
125 False True )
OUT peters-macbookpro.local():1
= < False True >
( < False True >
125 False True )
- = < False True >
( < False True >
125 False True )
- = < False True >
( < False True >
125 False True

In [16]:
print d1.getExitcode()

Should be one: one
125


In [17]:
print d1.d["stdout"]

Running test command on peters-macbookpro.local
peters-macbookpro.local():1
peters-macbookpro.local():2

